# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [9]:
problem = '''You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. 
'''

## 2. Ask for parameters

In [10]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [11]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

*  $i$: Index for thermal units, $i = 1, 2, ..., 6$
*  $t$: Index for time periods, $t = 1, 2, ..., 15$

## Parameters:

*  $N_{U}$: Number of thermal units ($N_U = 6$)
*  $N_{T}$: Number of time periods ($N_T = 15$)
*  $SU_i$: Startup cost for unit $i$ (€)
*  $SD_i$: Shutdown cost for unit $i$ (€)
*  $FC_i$: Fixed cost for running unit $i$ (€) 
*  $VC_{i,t}$: Variable cost for unit $i$ at time $t$ (€/MWh)
*  $P_{i}^{min}$: Minimum power output of unit $i$ (MW)
*  $P_{i}^{max}$: Maximum power output of unit $i$ (MW)
*  $RU_{i}$: Maximum power increment of unit $i$ (MW)
*  $RD_{i}$: Maximum power decrement of unit $i$ (MW) 
*  $D_t$: Total power demand at time $t$ (MW)

## Variables:

*  $P_{i,t}$: Power output of unit $i$ at time $t$ (MW)
*  $U_{i,t}$: Binary variable indicating if unit $i$ is running at time $t$ (1 if on, 0 if off)
*  $SU_{i,t}$: Binary variable indicating if unit $i$ is started up at time $t$ (1 if started, 0 otherwise)
*  $SD_{i,t}$: Binary variable indicating if unit $i$ is shut down at time $t$ (1 if shut down, 0 otherwise) 


# 2. Ask for objective

In [12]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [13]:
Markdown(response2.text)

$$
\text{Minimize} \sum_{i=1}^{N_U} \sum_{t=1}^{N_T} \left( SU_i \cdot SU_{i,t} + SD_i \cdot SD_{i,t} + FC_i \cdot U_{i,t} + VC_{i,t} \cdot P_{i,t} \right)
$$


# 3. Ask for constraints

In [14]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [15]:
Markdown(response3.text)

## Constraints:

**1. Power balance constraint:**

$$ \sum_{i=1}^{N_U} P_{i,t} \ge 1.1 \cdot D_t  \quad \forall t = 1, 2, ..., N_T$$

**2. Generation limits:**

$$ P_{i}^{min} \cdot U_{i,t} \le P_{i,t} \le P_{i}^{max} \cdot U_{i,t} \quad \forall i = 1, 2, ..., N_U; \forall t = 1, 2, ..., N_T$$

**3. Ramping up constraints:**

$$ P_{i,t} - P_{i,t-1} \le RU_i \quad \forall i = 1, 2, ..., N_U; \forall t = 2, 3, ..., N_T$$

**4. Ramping down constraints:**

$$ P_{i,t-1} - P_{i,t} \le RD_i \quad \forall i = 1, 2, ..., N_U;  \forall t = 2, 3, ..., N_T$$

**5. Startup and shutdown relationship:**

$$ SU_{i,t} \ge U_{i,t} - U_{i,t-1} \quad \forall i = 1, 2, ..., N_U;  \forall t = 2, 3, ..., N_T$$

$$ SD_{i,t} \ge U_{i,t-1} - U_{i,t} \quad \forall i = 1, 2, ..., N_U;  \forall t = 2, 3, ..., N_T$$ 

**6. Binary and non-negativity constraints:**

$$ U_{i,t}, SU_{i,t}, SD_{i,t} \in \{0, 1\} \quad \forall i = 1, 2, ..., N_U; \forall t = 1, 2, ..., N_T $$

$$ P_{i,t} \ge 0 \quad \forall i = 1, 2, ..., N_U; \forall t = 1, 2, ..., N_T$$


## 3. Generate the pyomo code

In [16]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [17]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data - replace with actual data
NU = 6  # Number of thermal units
NT = 15  # Number of time periods

# Cost parameters
SU = {1: 100, 2: 150, 3: 120, 4: 180, 5: 160, 6: 200}  # Startup cost (€)
SD = {1: 50, 2: 75, 3: 60, 4: 90, 5: 80, 6: 100}  # Shutdown cost (€)
FC = {1: 200, 2: 250, 3: 220, 4: 280, 5: 260, 6: 300}  # Fixed cost (€)
VC = {
    (i, t): 10 + i * 0.5 + t * 0.1
    for i in range(1, NU + 1)
    for t in range(1, NT + 1)
}  # Variable cost (€/MWh)

# Technical parameters
Pmin = {1: 50, 2: 80, 3: 60, 4: 100, 5: 90, 6: 120}  # Minimum power output (MW)
Pmax = {1: 200, 2: 300, 3: 250, 4: 350, 5: 320, 6: 400}  # Maximum power output (MW)
RU = {1: 80, 2: 100, 3: 90, 4: 120, 5: 110, 6: 140}  # Maximum ramp-up rate (MW/h)
RD = {1: 50, 2: 60, 3: 55, 4: 70, 5: 65, 6: 80}  # Maximum ramp-down rate (MW/h)

# Demand data
D = {t: 800 + t * 20 for t in range(1, NT + 1)}  # Total power demand (MW)


# Create a Concrete Model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.RangeSet(1, NU)  # Set of thermal units
model.T = pyo.RangeSet(1, NT)  # Set of time periods

# Define parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.Pmin = pyo.Param(model.I, initialize=Pmin)
model.Pmax = pyo.Param(model.I, initialize=Pmax)
model.RU = pyo.Param(model.I, initialize=RU)
model.RD = pyo.Param(model.I, initialize=RD)
model.D = pyo.Param(model.T, initialize=D)

# Define variables
model.P = pyo.Var(model.I, model.T, nonnegative=True)  # Power output (MW)
model.U = pyo.Var(
    model.I, model.T, pyo.Binary
)  # Unit commitment status (1=on, 0=off)
model.SU_var = pyo.Var(
    model.I, model.T, pyo.Binary
)  # Startup status (1=startup, 0=otherwise)
model.SD_var = pyo.Var(
    model.I, model.T, pyo.Binary
)  # Shutdown status (1=shutdown, 0=otherwise)

# Define objective function
def objective_function(model):
    return sum(
        model.SU[i] * model.SU_var[i, t]
        + model.SD[i] * model.SD_var[i, t]
        + model.FC[i] * model.U[i, t]
        + model.VC[i, t] * model.P[i, t]
        for i in model.I
        for t in model.T
    )


model.obj = pyo.Objective(rule=objective_function, sense=pyo.minimize)

# Define constraints
# 1. Power balance constraint
def power_balance(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * model.D[t]


model.power_balance = pyo.Constraint(model.T, rule=power_balance)

# 2. Generation limits
def generation_limits(model, i, t):
    return model.Pmin[i] * model.U[i, t] <= model.P[i, t] <= model.Pmax[i] * model.U[
        i, t
    ]


model.generation_limits = pyo.Constraint(model.I, model.T, rule=generation_limits)

# 3. Ramping up constraints
def ramping_up(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t - 1] <= model.RU[i]
    else:
        return pyo.Constraint.Skip


model.ramping_up = pyo.Constraint(model.I, model.T, rule=ramping_up)

# 4. Ramping down constraints
def ramping_down(model, i, t):
    if t > 1:
        return model.P[i, t - 1] - model.P[i, t] <= model.RD[i]
    else:
        return pyo.Constraint.Skip


model.ramping_down = pyo.Constraint(model.I, model.T, rule=ramping_down)

# 5. Startup and shutdown relationship
def startup_relationship(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.U[i, t] - model.U[i, t - 1]
    else:
        return pyo.Constraint.Skip


model.startup_relationship = pyo.Constraint(
    model.I, model.T, rule=startup_relationship
)


def shutdown_relationship(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.U[i, t - 1] - model.U[i, t]
    else:
        return pyo.Constraint.Skip


model.shutdown_relationship = pyo.Constraint(
    model.I, model.T, rule=shutdown_relationship
)


# Solve the model
solver = pyo.SolverFactory("cbc")  # Use CBC solver
solver.solve(model)

# Print the results
print("Total cost:", model.obj())
print("Unit commitment:")
for t in model.T:
    for i in model.I:
        print(f"Unit {i}, Time {t}: {model.U[i, t].value}")
print("Power output:")
for t in model.T:
    for i in model.I:
        print(f"Unit {i}, Time {t}: {model.P[i, t].value}")

```

I used sample data for the following parameters:

*   **SU, SD, FC:** Startup, Shutdown and fixed costs. I assumed some increasing costs for later units.
*   **VC:** Variable cost. I made it dependent on the unit and time. 
*   **Pmin, Pmax, RU, RD:** Minimum generation, maximum generation, ramp up and ramp down rates. I assumed some limits for each unit. 
*   **D:** Power demand. I used a simple example of linearly increasing demand. 

You can replace these with your actual data.

## 4. Input problem data and try running the generated code

In [18]:
import pyomo.environ as pyo

# Sample data - replace with actual data
NU = 6  # Number of thermal units
NT = 15  # Number of time periods

# Cost parameters
SU = {1: 100, 2: 150, 3: 120, 4: 180, 5: 160, 6: 200}  # Startup cost (€)
SD = {1: 50, 2: 75, 3: 60, 4: 90, 5: 80, 6: 100}  # Shutdown cost (€)
FC = {1: 200, 2: 250, 3: 220, 4: 280, 5: 260, 6: 300}  # Fixed cost (€)
VC = {
    (i, t): 10 + i * 0.5 + t * 0.1
    for i in range(1, NU + 1)
    for t in range(1, NT + 1)
}  # Variable cost (€/MWh)

# Technical parameters
Pmin = {1: 50, 2: 80, 3: 60, 4: 100, 5: 90, 6: 120}  # Minimum power output (MW)
Pmax = {1: 200, 2: 300, 3: 250, 4: 350, 5: 320, 6: 400}  # Maximum power output (MW)
RU = {1: 80, 2: 100, 3: 90, 4: 120, 5: 110, 6: 140}  # Maximum ramp-up rate (MW/h)
RD = {1: 50, 2: 60, 3: 55, 4: 70, 5: 65, 6: 80}  # Maximum ramp-down rate (MW/h)

# Demand data
D = {t: 800 + t * 20 for t in range(1, NT + 1)}  # Total power demand (MW)


# Create a Concrete Model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.RangeSet(1, NU)  # Set of thermal units
model.T = pyo.RangeSet(1, NT)  # Set of time periods

# Define parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.Pmin = pyo.Param(model.I, initialize=Pmin)
model.Pmax = pyo.Param(model.I, initialize=Pmax)
model.RU = pyo.Param(model.I, initialize=RU)
model.RD = pyo.Param(model.I, initialize=RD)
model.D = pyo.Param(model.T, initialize=D)

# Define variables
model.P = pyo.Var(model.I, model.T, nonnegative=True)  # Power output (MW)
model.U = pyo.Var(
    model.I, model.T, pyo.Binary
)  # Unit commitment status (1=on, 0=off)
model.SU_var = pyo.Var(
    model.I, model.T, pyo.Binary
)  # Startup status (1=startup, 0=otherwise)
model.SD_var = pyo.Var(
    model.I, model.T, pyo.Binary
)  # Shutdown status (1=shutdown, 0=otherwise)

# Define objective function
def objective_function(model):
    return sum(
        model.SU[i] * model.SU_var[i, t]
        + model.SD[i] * model.SD_var[i, t]
        + model.FC[i] * model.U[i, t]
        + model.VC[i, t] * model.P[i, t]
        for i in model.I
        for t in model.T
    )


model.obj = pyo.Objective(rule=objective_function, sense=pyo.minimize)

# Define constraints
# 1. Power balance constraint
def power_balance(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * model.D[t]


model.power_balance = pyo.Constraint(model.T, rule=power_balance)

# 2. Generation limits
def generation_limits(model, i, t):
    return model.Pmin[i] * model.U[i, t] <= model.P[i, t] <= model.Pmax[i] * model.U[
        i, t
    ]


model.generation_limits = pyo.Constraint(model.I, model.T, rule=generation_limits)

# 3. Ramping up constraints
def ramping_up(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t - 1] <= model.RU[i]
    else:
        return pyo.Constraint.Skip


model.ramping_up = pyo.Constraint(model.I, model.T, rule=ramping_up)

# 4. Ramping down constraints
def ramping_down(model, i, t):
    if t > 1:
        return model.P[i, t - 1] - model.P[i, t] <= model.RD[i]
    else:
        return pyo.Constraint.Skip


model.ramping_down = pyo.Constraint(model.I, model.T, rule=ramping_down)

# 5. Startup and shutdown relationship
def startup_relationship(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.U[i, t] - model.U[i, t - 1]
    else:
        return pyo.Constraint.Skip


model.startup_relationship = pyo.Constraint(
    model.I, model.T, rule=startup_relationship
)


def shutdown_relationship(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.U[i, t - 1] - model.U[i, t]
    else:
        return pyo.Constraint.Skip


model.shutdown_relationship = pyo.Constraint(
    model.I, model.T, rule=shutdown_relationship
)


# Solve the model
solver = pyo.SolverFactory("cbc")  # Use CBC solver
solver.solve(model)

# Print the results
print("Total cost:", model.obj())
print("Unit commitment:")
for t in model.T:
    for i in model.I:
        print(f"Unit {i}, Time {t}: {model.U[i, t].value}")
print("Power output:")
for t in model.T:
    for i in model.I:
        print(f"Unit {i}, Time {t}: {model.P[i, t].value}")

ValueError: Unexpected keyword options found while constructing 'IndexedVar':
	nonnegative

## 5. Correct the code to verify model viability (optional)

In [24]:
from pyomo.environ import *


import pyomo.environ as pyo

# Sample data - replace with actual data
NU = 6  # Number of thermal units
NT = 15  # Number of time periods

SU =  {'Plant1': 10324, 'Plant2': 5678, 'Plant3': 7802, 'Plant4': 12899, 'Plant5': 4596, 'Plant6': 9076}
SD = {'Plant1': 2673, 'Plant2': 5893, 'Plant3': 982, 'Plant4': 6783, 'Plant5': 2596, 'Plant6': 3561}
FC = {'Plant1': 2000, 'Plant2': 3000, 'Plant3': 2500, 'Plant4': 4000, 'Plant5': 3500, 'Plant6': 4500}  # Sample data
VC = {
    ('Plant1', 1): 20, ('Plant1', 2): 22, ('Plant1', 3): 23, ('Plant1', 4): 24, ('Plant1', 5): 25, 
    ('Plant1', 6): 26, ('Plant1', 7): 27, ('Plant1', 8): 28, ('Plant1', 9): 29, ('Plant1', 10): 30, 
    ('Plant1', 11): 31, ('Plant1', 12): 32, ('Plant1', 13): 33, ('Plant1', 14): 34, ('Plant1', 15): 35,

    ('Plant2', 1): 15, ('Plant2', 2): 16, ('Plant2', 3): 17, ('Plant2', 4): 18, ('Plant2', 5): 19, 
    ('Plant2', 6): 20, ('Plant2', 7): 21, ('Plant2', 8): 22, ('Plant2', 9): 23, ('Plant2', 10): 24, 
    ('Plant2', 11): 25, ('Plant2', 12): 26, ('Plant2', 13): 27, ('Plant2', 14): 28, ('Plant2', 15): 29,

    ('Plant3', 1): 18, ('Plant3', 2): 19, ('Plant3', 3): 20, ('Plant3', 4): 21, ('Plant3', 5): 22, 
    ('Plant3', 6): 23, ('Plant3', 7): 24, ('Plant3', 8): 25, ('Plant3', 9): 26, ('Plant3', 10): 27, 
    ('Plant3', 11): 28, ('Plant3', 12): 29, ('Plant3', 13): 30, ('Plant3', 14): 31, ('Plant3', 15): 32,

    ('Plant4', 1): 25, ('Plant4', 2): 26, ('Plant4', 3): 27, ('Plant4', 4): 28, ('Plant4', 5): 29, 
    ('Plant4', 6): 30, ('Plant4', 7): 31, ('Plant4', 8): 32, ('Plant4', 9): 33, ('Plant4', 10): 34, 
    ('Plant4', 11): 35, ('Plant4', 12): 36, ('Plant4', 13): 37, ('Plant4', 14): 38, ('Plant4', 15): 39,

    ('Plant5', 1): 22, ('Plant5', 2): 23, ('Plant5', 3): 24, ('Plant5', 4): 25, ('Plant5', 5): 26, 
    ('Plant5', 6): 27, ('Plant5', 7): 28, ('Plant5', 8): 29, ('Plant5', 9): 30, ('Plant5', 10): 31, 
    ('Plant5', 11): 32, ('Plant5', 12): 33, ('Plant5', 13): 34, ('Plant5', 14): 35, ('Plant5', 15): 36,

    ('Plant6', 1): 30, ('Plant6', 2): 31, ('Plant6', 3): 32, ('Plant6', 4): 33, ('Plant6', 5): 34, 
    ('Plant6', 6): 35, ('Plant6', 7): 36, ('Plant6', 8): 37, ('Plant6', 9): 38, ('Plant6', 10): 39, 
    ('Plant6', 11): 40, ('Plant6', 12): 41, ('Plant6', 13): 42, ('Plant6', 14): 43, ('Plant6', 15): 44
}

# Technical parameters
Pmin = {'Plant1': 50, 'Plant2': 40, 'Plant3': 30, 'Plant4': 60, 'Plant5': 55, 'Plant6': 65}  # Sample data
Pmax = {'Plant1': 500, 'Plant2': 600, 'Plant3': 550, 'Plant4': 700, 'Plant5': 650, 'Plant6': 750}  # Sample data
RU = {'Plant1': 100, 'Plant2': 120, 'Plant3': 110, 'Plant4': 130, 'Plant5': 125, 'Plant6': 140}  # Sample data
RD = {'Plant1': 90, 'Plant2': 110, 'Plant3': 100, 'Plant4': 120, 'Plant5': 115, 'Plant6': 130}  # Sample data
D =  {1: 280, 2: 327, 3: 400, 4: 388, 5: 501, 6: 600, 7: 800, 8: 927, 9: 705, 10: 502,
     11: 781, 12: 906, 13: 930, 14: 877, 15: 966}  # Sample data


# Create a Concrete Model
model = pyo.ConcreteModel()

# Define sets
I = ['Plant1', 'Plant2', 'Plant3', 'Plant4', 'Plant5', 'Plant6']
T = range(1, 16)

# Define sets
model.I = Set(initialize=I)
model.T = Set(initialize=T)

# Define parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.Pmin = pyo.Param(model.I, initialize=Pmin)
model.Pmax = pyo.Param(model.I, initialize=Pmax)
model.RU = pyo.Param(model.I, initialize=RU)
model.RD = pyo.Param(model.I, initialize=RD)
model.D = pyo.Param(model.T, initialize=D)

model.P = pyo.Var(model.I, model.T)  # Power output (MW)
model.U = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit commitment status (1=on, 0=off)
model.SU_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup status (1=startup, 0=otherwise)
model.SD_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown status (1=shutdown, 0=otherwise)

# Define objective function
def objective_function(model):
    return sum(
        model.SU[i] * model.SU_var[i, t]
        + model.SD[i] * model.SD_var[i, t]
        + model.FC[i] * model.U[i, t]
        + model.VC[i, t] * model.P[i, t]
        for i in model.I
        for t in model.T
    )

model.obj = pyo.Objective(rule=objective_function, sense=pyo.minimize)


#model.obj = pyo.Objective(rule=objective_function, sense=pyo.minimize)

# Define constraints
# 1. Power balance constraint
def power_balance(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * model.D[t]


model.power_balance = pyo.Constraint(model.T, rule=power_balance)

# 2. Generation limits
def generation_limits(model, i, t):
    return model.Pmin[i] * model.U[i, t] <= model.P[i, t] 

model.generation_limits = pyo.Constraint(model.I, model.T, rule=generation_limits)

# 2. Generation limits
def generation_limitsU(model, i, t):
    return model.P[i, t] <= model.Pmax[i] * model.U[
        i, t
    ]

model.generation_limitsU = pyo.Constraint(model.I, model.T, rule=generation_limitsU)

# 3. Ramping up constraints
def ramping_up(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t - 1] <= model.RU[i]
    else:
        return pyo.Constraint.Skip


model.ramping_up = pyo.Constraint(model.I, model.T, rule=ramping_up)

# 4. Ramping down constraints
def ramping_down(model, i, t):
    if t > 1:
        return model.P[i, t - 1] - model.P[i, t] <= model.RD[i]
    else:
        return pyo.Constraint.Skip


model.ramping_down = pyo.Constraint(model.I, model.T, rule=ramping_down)

# 5. Startup and shutdown relationship
def startup_relationship(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.U[i, t] - model.U[i, t - 1]
    else:
        return pyo.Constraint.Skip


model.startup_relationship = pyo.Constraint(
    model.I, model.T, rule=startup_relationship
)


def shutdown_relationship(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.U[i, t - 1] - model.U[i, t]
    else:
        return pyo.Constraint.Skip


model.shutdown_relationship = pyo.Constraint(
    model.I, model.T, rule=shutdown_relationship
)


# Solve the model
solver = pyo.SolverFactory("glpk")  # Use CBC solver
solver.solve(model)

# Print the results
print("Total cost:", model.obj())
print("Unit commitment:")
for t in model.T:
    for i in model.I:
        print(f"Unit {i}, Time {t}: {model.U[i, t].value}")
print("Power output:")
for t in model.T:
    for i in model.I:
        print(f"Unit {i}, Time {t}: {model.P[i, t].value}")

Total cost: 347586.9
Unit commitment:
Unit Plant1, Time 1: 0.0
Unit Plant2, Time 1: 1.0
Unit Plant3, Time 1: 0.0
Unit Plant4, Time 1: 0.0
Unit Plant5, Time 1: 0.0
Unit Plant6, Time 1: 0.0
Unit Plant1, Time 2: 0.0
Unit Plant2, Time 2: 1.0
Unit Plant3, Time 2: 0.0
Unit Plant4, Time 2: 0.0
Unit Plant5, Time 2: 0.0
Unit Plant6, Time 2: 0.0
Unit Plant1, Time 3: 0.0
Unit Plant2, Time 3: 1.0
Unit Plant3, Time 3: 0.0
Unit Plant4, Time 3: 0.0
Unit Plant5, Time 3: 0.0
Unit Plant6, Time 3: 0.0
Unit Plant1, Time 4: 0.0
Unit Plant2, Time 4: 1.0
Unit Plant3, Time 4: 0.0
Unit Plant4, Time 4: 0.0
Unit Plant5, Time 4: 0.0
Unit Plant6, Time 4: 0.0
Unit Plant1, Time 5: 0.0
Unit Plant2, Time 5: 1.0
Unit Plant3, Time 5: 1.0
Unit Plant4, Time 5: 0.0
Unit Plant5, Time 5: 0.0
Unit Plant6, Time 5: 0.0
Unit Plant1, Time 6: 0.0
Unit Plant2, Time 6: 1.0
Unit Plant3, Time 6: 1.0
Unit Plant4, Time 6: 0.0
Unit Plant5, Time 6: 0.0
Unit Plant6, Time 6: 0.0
Unit Plant1, Time 7: 0.0
Unit Plant2, Time 7: 1.0
Unit Plant3,

## 6. Print the responses

In [25]:
print(response.text)

## Indices:

*  $i$: Index for thermal units, $i = 1, 2, ..., 6$
*  $t$: Index for time periods, $t = 1, 2, ..., 15$

## Parameters:

*  $N_{U}$: Number of thermal units ($N_U = 6$)
*  $N_{T}$: Number of time periods ($N_T = 15$)
*  $SU_i$: Startup cost for unit $i$ (€)
*  $SD_i$: Shutdown cost for unit $i$ (€)
*  $FC_i$: Fixed cost for running unit $i$ (€) 
*  $VC_{i,t}$: Variable cost for unit $i$ at time $t$ (€/MWh)
*  $P_{i}^{min}$: Minimum power output of unit $i$ (MW)
*  $P_{i}^{max}$: Maximum power output of unit $i$ (MW)
*  $RU_{i}$: Maximum power increment of unit $i$ (MW)
*  $RD_{i}$: Maximum power decrement of unit $i$ (MW) 
*  $D_t$: Total power demand at time $t$ (MW)

## Variables:

*  $P_{i,t}$: Power output of unit $i$ at time $t$ (MW)
*  $U_{i,t}$: Binary variable indicating if unit $i$ is running at time $t$ (1 if on, 0 if off)
*  $SU_{i,t}$: Binary variable indicating if unit $i$ is started up at time $t$ (1 if started, 0 otherwise)
*  $SD_{i,t}$: Binary variable ind

In [26]:
print(response2.text)

$$
\text{Minimize} \sum_{i=1}^{N_U} \sum_{t=1}^{N_T} \left( SU_i \cdot SU_{i,t} + SD_i \cdot SD_{i,t} + FC_i \cdot U_{i,t} + VC_{i,t} \cdot P_{i,t} \right)
$$



In [27]:
print(response3.text)

## Constraints:

**1. Power balance constraint:**

$$ \sum_{i=1}^{N_U} P_{i,t} \ge 1.1 \cdot D_t  \quad \forall t = 1, 2, ..., N_T$$

**2. Generation limits:**

$$ P_{i}^{min} \cdot U_{i,t} \le P_{i,t} \le P_{i}^{max} \cdot U_{i,t} \quad \forall i = 1, 2, ..., N_U; \forall t = 1, 2, ..., N_T$$

**3. Ramping up constraints:**

$$ P_{i,t} - P_{i,t-1} \le RU_i \quad \forall i = 1, 2, ..., N_U; \forall t = 2, 3, ..., N_T$$

**4. Ramping down constraints:**

$$ P_{i,t-1} - P_{i,t} \le RD_i \quad \forall i = 1, 2, ..., N_U;  \forall t = 2, 3, ..., N_T$$

**5. Startup and shutdown relationship:**

$$ SU_{i,t} \ge U_{i,t} - U_{i,t-1} \quad \forall i = 1, 2, ..., N_U;  \forall t = 2, 3, ..., N_T$$

$$ SD_{i,t} \ge U_{i,t-1} - U_{i,t} \quad \forall i = 1, 2, ..., N_U;  \forall t = 2, 3, ..., N_T$$ 

**6. Binary and non-negativity constraints:**

$$ U_{i,t}, SU_{i,t}, SD_{i,t} \in \{0, 1\} \quad \forall i = 1, 2, ..., N_U; \forall t = 1, 2, ..., N_T $$

$$ P_{i,t} \ge 0 \quad \forall i = 1, 2, ..

In [28]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data - replace with actual data
NU = 6  # Number of thermal units
NT = 15  # Number of time periods

# Cost parameters
SU = {1: 100, 2: 150, 3: 120, 4: 180, 5: 160, 6: 200}  # Startup cost (€)
SD = {1: 50, 2: 75, 3: 60, 4: 90, 5: 80, 6: 100}  # Shutdown cost (€)
FC = {1: 200, 2: 250, 3: 220, 4: 280, 5: 260, 6: 300}  # Fixed cost (€)
VC = {
    (i, t): 10 + i * 0.5 + t * 0.1
    for i in range(1, NU + 1)
    for t in range(1, NT + 1)
}  # Variable cost (€/MWh)

# Technical parameters
Pmin = {1: 50, 2: 80, 3: 60, 4: 100, 5: 90, 6: 120}  # Minimum power output (MW)
Pmax = {1: 200, 2: 300, 3: 250, 4: 350, 5: 320, 6: 400}  # Maximum power output (MW)
RU = {1: 80, 2: 100, 3: 90, 4: 120, 5: 110, 6: 140}  # Maximum ramp-up rate (MW/h)
RD = {1: 50, 2: 60, 3: 55, 4: 70, 5: 65, 6: 80}  # Maximum ramp-down rate (MW/h)

# Demand data
D = {t: 800 + t * 20 for t in range(1, NT + 1)}  # Total power demand (MW)


# Create a Concrete Model
model = pyo.Conc